In [ ]:
### COMPUTATIONAL PHYSICS
### Homework 3, Problem 1: 2D Curve Fitting

# In class on 5 March 2019, we used Monte Carlo to fit a function to data with error bars in 
# two dimensions assuming a uniform random distribution of errors. Each of your was assigned a 
# different statistical error distribution (Sachi - Binominal, Carson - Possoin, Baylor - Gamma, 
# Christine - Log Normal, Hope - Gaussian, Sarah - Rayleigh). Your assignment is to create the
# following plots and answer the equations below.

# One one set of axis plot the following: 
# (1) your noisy data with the error bars in x and y, 
# (2) the function you inputting to create the data, 
# (3) the best fit curve from your Monte Carlo fitting, 
# (4) a shaded region showing the area between curves which are one standard deviation above 
# and below the best fit line. Make this plot for NMonteCarlo = 10,100,1000.

# How do your results for your given distribution compare to the ones you found in class for a 
# uniform distribution of errors?

# Comment on how your results might change with the intrinsic scatter and size of the error bars
# in your data. Did you need to populate the error bars to one standard deviation, two?

### Name:  Carson Huey-You

In [ ]:
### 1. IMPORT PACKAGES

import math as math
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

In [ ]:
### 2. DEFINE FUNCTIONS

# Define a Generating Function, which we will use to create the noisy data.

def guess_func(x, a, b, c):
    return a*np.sin(b*x + c)



In [ ]:
# Iterate through a monte-carlo fitting scheme N_mc times, 
# and return value_count values.

def montecarlo_fit(guess_func, x_data, y_data, x_err, y_err, N_mc, value_count):
    
    #Get number of data points to fit to.
    N = len(x_data)
    
    #Define array to save optimized values.
    saved_vals = np.zeros(shape=(value_count, N_mc))
    
    #Populate 'error boxes'.
    for i in range(0, N_mc):
        
        x_mc = x_data + (np.random.random(N) - 0.5)*2.0*x_err
        y_mc = y_data + (np.random.random(N) - 0.5)*2.0*y_err
        
        popt, pcov = curve_fit(guess_func, x_mc, y_mc)
        
        #Save each popt in an array.
        for j in range(0, value_count):
            
            saved_vals[j][i] = popt[j]
            
    #Derive median & stdev values for a, b, and c.
    guess_a = np.median(saved_vals[0])
    guess_b = np.median(saved_vals[1])
    guess_c = np.median(saved_vals[2])
    
    stdev_a = np.std(saved_vals[0])
    stdev_b = np.std(saved_vals[1])
    stdev_c = np.std(saved_vals[2])
    
    return np.array([[guess_a, guess_b, guess_c], 
                     [stdev_a, stdev_b, stdev_c]])



In [ ]:
# Add multiple elements to graphs, specifically:
# - Scattered Data with Error-Bars
# - Best Fit from Monte-Carlo
# - Filled Area ±K Standard Deviations from the Best Fit

def plot_stuff(guess_func, x_data, y_data, x_err, y_err, N_mc, value_count):
    
    #Scattered Data with Error-Bars.
    plt.errorbar(x_data, y_data, xerr=x_err, yerr=y_err, fmt='o', markersize=2, alpha=0.5, label="Noisy Data to Fit")
    
    #Best Fit from Monte-Carlo.
    value_array = montecarlo_fit(guess_func, x_data, y_data, x_err, y_err, N_mc, value_count)
    
    guess_a, guess_b, guess_c = value_array[0][0], value_array[0][1], value_array[0][2]
    stdev_a, stdev_b, stdev_c = value_array[1][0], value_array[1][1], value_array[1][2]
    
    fit_label = f"\nBest Fit from Monte-Carlo:\n"
    fit_a = f"a={guess_a}, " + "$\sigma_{a}=$" + f"{stdev_a}\n" 
    fit_b = f"b={guess_b}, " + "$\sigma_{b}=$" + f"{stdev_b}\n"
    fit_c = f"c={guess_c}, " + "$\sigma_{c}=$" + f"{stdev_c}"
    
    plt.plot(x_data, guess_func(x_data, guess_a, guess_b, guess_c), color='yellow', label = fit_label + fit_a + fit_b + fit_c)
    
    #Filled Area ±K Standard Deviations from the Best Fit.
    #Change how many standard deviations to fill with stretch_fill.
    stretch_fill = 1
    
    top_fill = guess_func(x_data, guess_a + stretch_fill*stdev_a, guess_b + stretch_fill*stdev_b, guess_c + stretch_fill*stdev_c)
    bot_fill = guess_func(x_data, guess_a - stretch_fill*stdev_a, guess_b - stretch_fill*stdev_b, guess_c - stretch_fill*stdev_c)
    
    plt.fill_between(x_data, top_fill, bot_fill, color='green', alpha=0.75, label="One Standard Deviation ($\pm$)")



In [ ]:
### 3. GENERATE DATA

N = 501

#Assume:
a, b, c = 2, 1, 3 #Y = 2*sin(x + 3)

#Number of values to find.
value_count = 3

x = np.linspace(0, 10, N)
y = guess_func(x, a, b, c) 

#Get errorbars for x and y.
error_x = 0.25
error_y = 0.4
x_err = error_x * np.random.poisson(lam=1,size=N)
y_err = error_y * np.random.poisson(lam=1,size=N)

#Scatter values with noise parameter.
noise=3
y_noise = y + 2*noise*(np.random.random(N) - 0.5)


In [ ]:
#Test plot, to show generating function, noise, and error.

fig = plt.figure(figsize=(10,10))

plt.plot(x, y, color='red', linestyle=':')

plt.scatter(x, y_noise, s=10, color='green', marker='s')
plt.errorbar(x, y_noise, xerr=x_err, yerr=y_err, fmt='o', markersize=1, alpha=0.5)


In [ ]:
### 4. FIT & PLOT DATA: N_mc=10

N_mc = 10

fig = plt.figure(figsize=(15,10))

plot_title = "Monte Carlo Data Fit with $N_{mc}$ = " + f"{N_mc} | " + f"F(x)=a*sin(bx+c)"
plt.title(plot_title, size=14)

plt.plot(x, y, color='red', linestyle=':', label=f"Generating Function: a={a}, b={b}, c={c}.")

plot_stuff(guess_func, x, y_noise, x_err, y_err, N_mc, 3)

plt.legend()

In [ ]:
### 4. FIT & PLOT DATA: N_mc=100

N_mc = 100

fig = plt.figure(figsize=(15,10))

plot_title = "Monte Carlo Data Fit with $N_{mc}$ = " + f"{N_mc} | " + f"F(x)=a*sin(bx+c)"
plt.title(plot_title, size=14)

plt.plot(x, y, color='red', linestyle=':', label=f"Generating Function: a={a}, b={b}, c={c}")

plot_stuff(guess_func, x, y_noise, x_err, y_err, N_mc, 3)

plt.legend()

In [ ]:
### 4. FIT & PLOT DATA: N_mc=1000

N_mc = 1000

fig = plt.figure(figsize=(15,10))

plot_title = "Monte Carlo Data Fit with $N_{mc}$ = " + f"{N_mc} | " + f"F(x)=a*sin(bx+c)"
plt.title(plot_title, size=14)

plt.plot(x, y, color='red', linestyle=':', label=f"Generating Function: a={a}, b={b}, c={c}")

plot_stuff(guess_func, x, y_noise, x_err, y_err, N_mc, 3)

plt.legend()